In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import os.path
import skimage
import skimage.segmentation
import sklearn.preprocessing
import sklearn.model_selection
import math
import shutil
import pathlib
import glob
import shutil
import uuid
import random
import platform
import torch
import torchvision
import numpy as np
import scipy as sp
import scipy.io
import scipy.signal
import pandas as pd
import networkx
import wfdb
import json
import tqdm
import dill
import pickle
import matplotlib.pyplot as plt

import scipy.stats
import cv2

import src.data
import src.metrics
import sak
import sak.signal.wavelet
import sak.data
import sak.data.augmentation
import sak.data.preprocessing
import sak.visualization
import sak.visualization.signal
import sak.torch
import sak.torch.nn
import sak.torch.nn as nn
import sak.torch.nn
import sak.torch.train
import sak.torch.data
import sak.torch.models
import sak.torch.models.lego
import sak.torch.models.variational
import sak.torch.models.classification

from sak.signal import StandardHeader

# Read all common files

In [3]:
basedir = '/media/guille/DADES/DADES/Delineator/'

with open('../list_files.txt') as f:
    valid_models = f.read().splitlines()
    
folders = [os.path.join(basedir,'TrainedModels',model) for model in valid_models]
all_databases = ["qtdb_single","qtdb_multi","zhejiang","ludb"]
all_waves = ["P","QRS","T"]
all_12_leads = ["zhejiang", "ludb"]
all_qtdb = ["qtdb_single","qtdb_multi"]

# Read and homoginize results - extract

In [4]:
zhejiang = {}
ludb = {}
qtdb_single = {}
qtdb_multi = {}
qtdb_fold_single = {}
qtdb_fold_multi = {}

for counter,folder in tqdm.tqdm(enumerate(folders)):
    if (
        (not os.path.isfile(os.path.join(folder,'metrics_string.txt'))) or
        (not os.path.isfile(os.path.join(folder,'zhejiang_metrics_string.txt'))) or
        (not os.path.isfile(os.path.join(folder,'ludb_metrics_string.txt'))) or
        (not os.path.isfile(os.path.join(folder,'metrics_fold_string.txt')))
       ):
        continue
    
    # Extract model name from folder path
    root,model_name = os.path.split(folder)
    
    # Init metrics
    zhejiang[model_name] = {'P': {}, 'QRS': {}, 'T': {}}
    ludb[model_name] = {'P': {}, 'QRS': {}, 'T': {}}
    qtdb_single[model_name] = {'P': {}, 'QRS': {}, 'T': {}}
    qtdb_multi[model_name] = {'P': {}, 'QRS': {}, 'T': {}}
    qtdb_fold_single[model_name] = {'P': {}, 'QRS': {}, 'T': {}}
    qtdb_fold_multi[model_name] = {'P': {}, 'QRS': {}, 'T': {}}
    
    # Open metrics files
    with open(os.path.join(folder,'metrics_string.txt')) as f:
        metrics = f.read().splitlines()
    with open(os.path.join(folder,'metrics_fold_string.txt')) as f:
        metrics_fold = f.read().splitlines()
    with open(os.path.join(folder,'zhejiang_metrics_string.txt')) as f:
        metrics_zhejiang = f.read().splitlines()
    with open(os.path.join(folder,'ludb_metrics_string.txt')) as f:
        metrics_ludb = f.read().splitlines()
    
    # Agglomerate Zhejiang's and LUDB's metrics
    for i,db in enumerate(all_12_leads):
        dic = eval(db)
        mtr = eval(f"metrics_{db}")
        
        for j,wave in enumerate(all_waves):
            dic[model_name][wave]["Precision"] = float(mtr[4+j*10].split(':')[1][:-1])
            dic[model_name][wave]["Recall"]    = float(mtr[5+j*10].split(':')[1][:-1])
            dic[model_name][wave]["F1 Score"]  = float(mtr[6+j*10].split(':')[1][:-1])

            onmstd = mtr[8+j*10].split(':')[1][:-2]
            offmstd = mtr[9+j*10].split(':')[1][:-2]
            dic[model_name][wave]["Onset Error (Mean)"] = float(onmstd.split('±')[0])
            dic[model_name][wave]["Onset Error (STD)"] = float(onmstd.split('±')[1])
            dic[model_name][wave]["Offset Error (Mean)"] = float(offmstd.split('±')[0])
            dic[model_name][wave]["Offset Error (STD)"] = float(offmstd.split('±')[1])
    
    # Extract QTDB single and multi
    for i,type in enumerate(all_qtdb):
        dic = eval(type)
        
        for j,wave in enumerate(all_waves):
            dic[model_name][wave]["Precision"] = float(metrics[(34*i)+(7+j*10)].split(':')[1][:-1])
            dic[model_name][wave]["Recall"]    = float(metrics[(34*i)+(8+j*10)].split(':')[1][:-1])
            dic[model_name][wave]["F1 Score"]  = float(metrics[(34*i)+(9+j*10)].split(':')[1][:-1])

            onmstd = metrics[(34*i)+(11+j*10)].split(':')[1][:-2]
            offmstd = metrics[(34*i)+(12+j*10)].split(':')[1][:-2]
            dic[model_name][wave]["Onset Error (Mean)"] = float(onmstd.split('±')[0])
            dic[model_name][wave]["Onset Error (STD)"] = float(onmstd.split('±')[1])
            dic[model_name][wave]["Offset Error (Mean)"] = float(offmstd.split('±')[0])
            dic[model_name][wave]["Offset Error (STD)"] = float(offmstd.split('±')[1])


109it [00:03, 30.20it/s]


# Analyze results - best F1 score overall

In [5]:
P_F1   = {}
QRS_F1 = {}
T_F1   = {}

for wave in ["P", "QRS", "T"]:
    output = eval(f"{wave}_F1")
    
    for db in ["qtdb_single","qtdb_multi","zhejiang","ludb"]:
        database = eval(db)
        if db not in output:
            output[db] = {}

        for k in database:
            output[db][k] = database[k][wave]["F1 Score"]
            
# Convert to DataFrame
P_F1   = pd.DataFrame(P_F1)
QRS_F1 = pd.DataFrame(QRS_F1)
T_F1   = pd.DataFrame(T_F1)

# Mean results across all databases and waves
F1_overall = (P_F1.mean(axis=1)+QRS_F1.mean(axis=1)+T_F1.mean(axis=1))/3

# Display results
print(("BEST ", F1_overall.idxmax(), F1_overall.max()))
print(("WORST", F1_overall.idxmin(), F1_overall.min()))
print("\n")

# Per wave
for wave in all_waves:
    wv = eval(f"{wave}_F1")
    print(f"########## {wave} wave ##########")
    print(("BEST ", wv.mean(axis=1).idxmax(), wv.mean(axis=1).max()))
    print(("WORST", wv.mean(axis=1).idxmin(), wv.mean(axis=1).min()))
    print("\n")


('BEST ', 'UNet7LevelsConvDiceOnly_20201203124825', 99.38250000000001)
('WORST', 'UNet7LevelsDiceOnly_20201202071040', 95.16250000000001)


########## P wave ##########
('BEST ', 'WNet5LevelsSelfAttentionDiceOnly_20201130125349', 98.99000000000001)
('WORST', 'UNet7LevelsDiceOnly_20201202071040', 85.98750000000001)


########## QRS wave ##########
('BEST ', 'UNet7Levels_synth_20201208095004', 99.8475)
('WORST', 'WNet5LevelsSelfAttention_synth_20201206012731', 99.35749999999999)


########## T wave ##########
('BEST ', 'UNet7LevelsDiceOnly_20201202071040', 99.69)
('WORST', 'UNet5LevelsConvDiceOnly_real_20201204204042', 96.95750000000001)




# Analyze results - minimum STD

In [6]:
P_STD_on    = {}
P_STD_off   = {}
QRS_STD_on  = {}
QRS_STD_off = {}
T_STD_on    = {}
T_STD_off   = {}

for wave in ["P", "QRS", "T"]:
    output_on  = eval(f"{wave}_STD_on")
    output_off = eval(f"{wave}_STD_off")
    
    for db in ["qtdb_single","qtdb_multi","zhejiang","ludb"]:
        database = eval(db)
        if db not in output_on:
            output_on[db] = {}
        if db not in output_off:
            output_off[db] = {}

        for k in database:
            output_on[db][k] = database[k][wave]["Onset Error (STD)"]
            output_off[db][k] = database[k][wave]["Offset Error (STD)"]

# Convert to DataFrame
P_STD_on    = pd.DataFrame(P_STD_on)
P_STD_off   = pd.DataFrame(P_STD_off)
QRS_STD_on  = pd.DataFrame(QRS_STD_on)
QRS_STD_off = pd.DataFrame(QRS_STD_off)
T_STD_on    = pd.DataFrame(T_STD_on)
T_STD_off   = pd.DataFrame(T_STD_off)

# Mean results across all databases and waves
onoff_overall = (
    ((P_STD_on+P_STD_off)/2).mean(axis=1)+
    ((QRS_STD_on+QRS_STD_off)/2).mean(axis=1)+
    ((T_STD_on+T_STD_off)/2).mean(axis=1))/3

# Display results
print(("BEST ", onoff_overall.idxmin(), onoff_overall.min()))
print(("WORST", onoff_overall.idxmax(), onoff_overall.max()))
print("\n")

# Per wave
for wave in all_waves:
    wv_on = eval(f"{wave}_STD_on")
    wv_off = eval(f"{wave}_STD_off")
    print(f"########## {wave} wave ##########")
    print(("BEST ", ((wv_on+wv_off)/2).mean(axis=1).idxmin(), ((wv_on+wv_off)/2).mean(axis=1).min()))
    print(("WORST", ((wv_on+wv_off)/2).mean(axis=1).idxmax(), ((wv_on+wv_off)/2).mean(axis=1).max()))
    print("\n")


('BEST ', 'WNet5LevelsSelfAttentionDiceOnly_20201130125349', 18.232083333333332)
('WORST', 'UNet5LevelsConvDiceOnly_20201202003451', 56.17374999999999)


########## P wave ##########
('BEST ', 'WNet5LevelsSelfAttentionConvDiceOnly_20201130125349', 12.90875)
('WORST', 'UNet5LevelsConvDiceOnly_20201202003451', 41.68)


########## QRS wave ##########
('BEST ', 'WNet5LevelsSelfAttentionDiceOnly_20201130125349', 10.479999999999999)
('WORST', 'UNet5LevelsConvDiceOnly_20201202003451', 38.32)


########## T wave ##########
('BEST ', 'UNet7LevelsConvDiceOnly_20201203124825', 30.746250000000003)
('WORST', 'UNet5LevelsConvDiceOnly_20201202003451', 88.52124999999998)




# Real vs synthetic vs both

In [7]:
real  = [model for model in valid_models if ("_real_" in model)]
synth = [model for model in valid_models if ("_synth_" in model)]
both  = [model for model in valid_models if ((model not in real) and (model not in synth))]

In [8]:
metric = "F1 Score"

real_as_metric = {}
synth_as_metric = {}
both_as_metric = {}

for db in all_databases:
    database = eval(db)
    
    for k in database:
        value = {wave: database[k][wave][metric] for wave in all_waves}
        unikey = k.split('_')[0]
        
        if k in real:
            value = real_as_metric.get(unikey,{})
            value.update({f"{wave}_{db}": database[k][wave][metric] for wave in all_waves})
            real_as_metric[unikey] = value
        elif k in synth:
            value = synth_as_metric.get(unikey,{})
            value.update({f"{wave}_{db}": database[k][wave][metric] for wave in all_waves})
            synth_as_metric[unikey] = value
        elif k in both:
            value = both_as_metric.get(unikey,{})
            value.update({f"{wave}_{db}": database[k][wave][metric] for wave in all_waves})
            both_as_metric[unikey] = value
        else:
            raise ValueError("This should not have happened")

# Convert to DataFrame
real_as_metric = pd.DataFrame(real_as_metric)
synth_as_metric = pd.DataFrame(synth_as_metric)
both_as_metric = pd.DataFrame(both_as_metric)

# Drop faulty training
if "UNet7LevelsDiceOnly" in real_as_metric:
    real_as_metric = real_as_metric.T.drop("UNet7LevelsDiceOnly").drop("UNet6LevelsDiceOnly").T
if "UNet7LevelsDiceOnly" in synth_as_metric:
    synth_as_metric = synth_as_metric.T.drop("UNet7LevelsDiceOnly").drop("UNet6LevelsDiceOnly").T
if "UNet7LevelsDiceOnly" in both_as_metric:
    both_as_metric = both_as_metric.T.drop("UNet7LevelsDiceOnly").drop("UNet6LevelsDiceOnly").T

# Display results
print(("BOTH VS  REAL", (both_as_metric.T-real_as_metric.T).mean(axis=1).mean()))
print(("BOTH VS SYNTH", (both_as_metric.T-synth_as_metric.T).mean(axis=1).mean()))
print(("SYNTH VS REAL", (synth_as_metric.T-real_as_metric.T).mean(axis=1).mean()))
print("\n")

# Per wave
for wave in all_waves:
    real_mtr = eval(f"real_as_metric").T[[f'{wave}_qtdb_single',f'{wave}_qtdb_multi',f'{wave}_ludb',f'{wave}_zhejiang',]].T
    synth_mtr = eval(f"synth_as_metric").T[[f'{wave}_qtdb_single',f'{wave}_qtdb_multi',f'{wave}_ludb',f'{wave}_zhejiang',]].T
    both_mtr = eval(f"both_as_metric").T[[f'{wave}_qtdb_single',f'{wave}_qtdb_multi',f'{wave}_ludb',f'{wave}_zhejiang',]].T
    print(f"########## {wave} wave ##########")
    print(("BOTH VS  REAL", (both_mtr.T-real_mtr.T).mean(axis=1).mean()))
    print(("BOTH VS SYNTH", (both_mtr.T-synth_mtr.T).mean(axis=1).mean()))
    print(("SYNTH VS REAL", (synth_mtr.T-real_mtr.T).mean(axis=1).mean()))
    print("\n")


('BOTH VS  REAL', 0.6252500000000002)
('BOTH VS SYNTH', 0.2774583333333335)
('SYNTH VS REAL', 0.34779166666666683)


########## P wave ##########
('BOTH VS  REAL', 1.0267500000000012)
('BOTH VS SYNTH', 0.7138750000000009)
('SYNTH VS REAL', 0.31287500000000035)


########## QRS wave ##########
('BOTH VS  REAL', 0.07800000000000082)
('BOTH VS SYNTH', -0.0025000000000000356)
('SYNTH VS REAL', 0.08050000000000086)


########## T wave ##########
('BOTH VS  REAL', 0.7709999999999985)
('BOTH VS SYNTH', 0.12099999999999937)
('SYNTH VS REAL', 0.6499999999999991)




# U-Net vs W-Net

In [9]:
metric = "F1 Score"

unet = [model for model in valid_models if ("UNet" in model)]
wnet = [model for model in valid_models if ("WNet" in model)]

unet_as_metric = {}
wnet_as_metric = {}

for db in all_databases:
    database = eval(db)
    
    for k in database:
        value = {wave: database[k][wave][metric] for wave in all_waves}
        unikey = "_".join(k.replace("UNet","").replace("WNet","").split("_")[:-1])
        
        if k in unet:
            value = unet_as_metric.get(unikey,{})
            value.update({f"{wave}_{db}": database[k][wave][metric] for wave in all_waves})
            unet_as_metric[unikey] = value
        elif k in wnet:
            value = wnet_as_metric.get(unikey,{})
            value.update({f"{wave}_{db}": database[k][wave][metric] for wave in all_waves})
            wnet_as_metric[unikey] = value
        else:
            raise ValueError("This should not have happened")

# Convert to DataFrame
unet_as_metric = pd.DataFrame(unet_as_metric)
wnet_as_metric = pd.DataFrame(wnet_as_metric)

# Drop faulty training
if "UNet7LevelsDiceOnly" in unet_as_metric:
    unet_as_metric = unet_as_metric.T.drop("UNet7LevelsDiceOnly").drop("UNet6LevelsDiceOnly").T
if "UNet7LevelsDiceOnly" in wnet_as_metric:
    wnet_as_metric = wnet_as_metric.T.drop("UNet7LevelsDiceOnly").drop("UNet6LevelsDiceOnly").T

# Display results
print(("WNET VS UNET", (wnet_as_metric.T-unet_as_metric.T).mean(axis=1).mean()))
print("\n")

# Per wave
for wave in all_waves:
    unet_mtr = eval(f"unet_as_metric").T[[f'{wave}_qtdb_single',f'{wave}_qtdb_multi',f'{wave}_ludb',f'{wave}_zhejiang',]].T
    wnet_mtr = eval(f"wnet_as_metric").T[[f'{wave}_qtdb_single',f'{wave}_qtdb_multi',f'{wave}_ludb',f'{wave}_zhejiang',]].T
    print(f"########## {wave} wave ##########")
    print(("WNET VS UNET", (wnet_mtr.T-unet_mtr.T).mean(axis=1).mean()))
    print("\n")


('WNET VS UNET', 0.5127777777777768)


########## P wave ##########
('WNET VS UNET', 0.7370833333333321)


########## QRS wave ##########
('WNET VS UNET', 0.10319444444444395)


########## T wave ##########
('WNET VS UNET', 0.6980555555555544)




# Dice vs F1

In [10]:
metric = "F1 Score"

dice = [model for model in valid_models if ("DiceOnly" in model)]
f1sc = [model for model in valid_models if ("DiceOnly" not in model)]

dice_as_metric = {}
f1sc_as_metric = {}

for db in all_databases:
    database = eval(db)
    
    for k in database:
        value = {wave: database[k][wave][metric] for wave in all_waves}
        unikey = "_".join(k.replace("DiceOnly","").split("_")[:-1])
        
        if k in dice:
            value = dice_as_metric.get(unikey,{})
            value.update({f"{wave}_{db}": database[k][wave][metric] for wave in all_waves})
            dice_as_metric[unikey] = value
        elif k in f1sc:
            value = f1sc_as_metric.get(unikey,{})
            value.update({f"{wave}_{db}": database[k][wave][metric] for wave in all_waves})
            f1sc_as_metric[unikey] = value
        else:
            raise ValueError("This should not have happened")

# Convert to DataFrame
dice_as_metric = pd.DataFrame(dice_as_metric)
f1sc_as_metric = pd.DataFrame(f1sc_as_metric)

# Display results
print(("f1sc VS dice", (f1sc_as_metric.T-dice_as_metric.T).mean(axis=1).mean()))
print("\n")
print("\n")
print("\n")

# Per wave
for wave in all_waves:
    dice_mtr = dice_as_metric.T[[f'{wave}_qtdb_single',f'{wave}_qtdb_multi',f'{wave}_ludb',f'{wave}_zhejiang',]].T
    f1sc_mtr = f1sc_as_metric.T[[f'{wave}_qtdb_single',f'{wave}_qtdb_multi',f'{wave}_ludb',f'{wave}_zhejiang',]].T
    print(f"########## {wave} wave ##########")
    print(("f1sc VS dice", (f1sc_mtr.T-dice_mtr.T).mean(axis=1).mean()))
    print("\n")

print("\n")
print("\n")

# Per database
for db in all_databases:
    dice_mtr = dice_as_metric.T[[f"P_{db}",f"QRS_{db}",f"T_{db}"]].T
    f1sc_mtr = f1sc_as_metric.T[[f"P_{db}",f"QRS_{db}",f"T_{db}"]].T
    print(f"########## {db} database ##########")
    print(("f1sc VS dice", (f1sc_mtr.T-dice_mtr.T).mean(axis=1).mean()))
    print("\n")


('f1sc VS dice', 0.25262626262626253)






########## P wave ##########
('f1sc VS dice', 0.38643939393939364)


########## QRS wave ##########
('f1sc VS dice', 0.031363636363636475)


########## T wave ##########
('f1sc VS dice', 0.3400757575757576)






########## qtdb_single database ##########
('f1sc VS dice', 0.5213131313131317)


########## qtdb_multi database ##########
('f1sc VS dice', 0.5013131313131314)


########## zhejiang database ##########
('f1sc VS dice', 0.08898989898989831)


########## ludb database ##########
('f1sc VS dice', -0.10111111111111115)




# SelfAttention vs Not

In [11]:
metric = "F1 Score"

selfatt = [model for model in valid_models if ("SelfAttentionConv" in model)]
noselfatt = [model for model in valid_models if ("SelfAttentionConv" not in model)]

selfatt_as_metric = {}
noselfatt_as_metric = {}

for db in all_databases:
    database = eval(db)
    
    for k in database:
        value = {wave: database[k][wave][metric] for wave in all_waves}
        unikey = "_".join(k.replace("SelfAttentionConv","").split("_")[:-1])
        
        if k in selfatt:
            value = selfatt_as_metric.get(unikey,{})
            value.update({f"{wave}_{db}": database[k][wave][metric] for wave in all_waves})
            selfatt_as_metric[unikey] = value
        elif k in noselfatt:
            value = noselfatt_as_metric.get(unikey,{})
            value.update({f"{wave}_{db}": database[k][wave][metric] for wave in all_waves})
            noselfatt_as_metric[unikey] = value
        else:
            raise ValueError("This should not have happened")

# Convert to DataFrame
selfatt_as_metric = pd.DataFrame(selfatt_as_metric)
noselfatt_as_metric = pd.DataFrame(noselfatt_as_metric)

# Display results
print(("noselfatt VS selfatt", (noselfatt_as_metric.T-selfatt_as_metric.T).mean(axis=1).mean()))
print("\n")
print("\n")
print("\n")

# Per wave
for wave in all_waves:
    selfatt_mtr = selfatt_as_metric.T[[f'{wave}_qtdb_single',f'{wave}_qtdb_multi',f'{wave}_ludb',f'{wave}_zhejiang',]].T
    noselfatt_mtr = noselfatt_as_metric.T[[f'{wave}_qtdb_single',f'{wave}_qtdb_multi',f'{wave}_ludb',f'{wave}_zhejiang',]].T
    print(f"########## {wave} wave ##########")
    print(("noselfatt VS selfatt", (noselfatt_mtr.T-selfatt_mtr.T).mean(axis=1).mean()))
    print("\n")

print("\n")
print("\n")

# Per database
for db in all_databases:
    selfatt_mtr = selfatt_as_metric.T[[f"P_{db}",f"QRS_{db}",f"T_{db}"]].T
    noselfatt_mtr = noselfatt_as_metric.T[[f"P_{db}",f"QRS_{db}",f"T_{db}"]].T
    print(f"########## {db} database ##########")
    print(("noselfatt VS selfatt", (noselfatt_mtr.T-selfatt_mtr.T).mean(axis=1).mean()))
    print("\n")


('noselfatt VS selfatt', 0.08263888888888853)






########## P wave ##########
('noselfatt VS selfatt', 0.06291666666666688)


########## QRS wave ##########
('noselfatt VS selfatt', 0.08333333333333155)


########## T wave ##########
('noselfatt VS selfatt', 0.10166666666666717)






########## qtdb_single database ##########
('noselfatt VS selfatt', 0.11111111111110876)


########## qtdb_multi database ##########
('noselfatt VS selfatt', 0.1772222222222237)


########## zhejiang database ##########
('noselfatt VS selfatt', 0.1494444444444443)


########## ludb database ##########
('noselfatt VS selfatt', -0.10722222222222261)




# Levels

In [12]:
valid_models

['UNet5LevelsConvDiceOnly_20201202003451',
 'UNet5LevelsConvDiceOnly_real_20201204204042',
 'UNet5LevelsConv_20201126031519',
 'UNet5LevelsConv_real_20201128082143',
 'UNet5LevelsConv_synth_20201207090502',
 'UNet5LevelsDiceOnly_20201130125350',
 'UNet5LevelsDiceOnly_real_20201203140435',
 'UNet5LevelsDiceOnly_synth_20201209121447',
 'UNet5Levels_20201124143433',
 'UNet5Levels_real_20201127152623',
 'UNet5Levels_synth_20201205163610',
 'UNet6LevelsConvDiceOnly_20201202055324',
 'UNet6LevelsConvDiceOnly_real_20201205031809',
 'UNet6LevelsConv_20201126052228',
 'UNet6LevelsConv_real_20201128200118',
 'UNet6LevelsConv_synth_20201208041815',
 'UNet6LevelsDiceOnly_20201202004750',
 'UNet6LevelsDiceOnly_real_20201205001620',
 'UNet6Levels_20201126042437',
 'UNet6Levels_real_20201128150827',
 'UNet6Levels_synth_20201207160352',
 'UNet7LevelsConvDiceOnly_20201203124825',
 'UNet7LevelsConvDiceOnly_real_20201205081529',
 'UNet7LevelsConv_20201127152515',
 'UNet7LevelsConv_real_20201129035943',
 

In [13]:
metric = "F1 Score"

level5 = [model for model in valid_models if ("5Levels" in model)]
level6 = [model for model in valid_models if ("6Levels" in model)]
level7 = [model for model in valid_models if ("7Levels" in model)]


level5_as_metric = {}
level6_as_metric = {}
level7_as_metric = {}

for db in all_databases:
    database = eval(db)
    
    for k in database:
        value = {wave: database[k][wave][metric] for wave in all_waves}
        unikey = "_".join(k.split('_')[:-1]).replace("5Levels","").replace("6Levels","").replace("7Levels","")
        
        if k in level5:
            value = level5_as_metric.get(unikey,{})
            value.update({f"{wave}_{db}": database[k][wave][metric] for wave in all_waves})
            level5_as_metric[unikey] = value
        elif k in level6:
            value = level6_as_metric.get(unikey,{})
            value.update({f"{wave}_{db}": database[k][wave][metric] for wave in all_waves})
            level6_as_metric[unikey] = value
        elif k in level7:
            value = level7_as_metric.get(unikey,{})
            value.update({f"{wave}_{db}": database[k][wave][metric] for wave in all_waves})
            level7_as_metric[unikey] = value
        else:
            raise ValueError("This should not have happened")

# Convert to DataFrame
level5_as_metric = pd.DataFrame(level5_as_metric)
level6_as_metric = pd.DataFrame(level6_as_metric)
level7_as_metric = pd.DataFrame(level7_as_metric)

# Display results
print(("level7 VS level5", (level7_as_metric.T-level5_as_metric.T).mean(axis=1).mean()))
print(("level7 VS level6", (level7_as_metric.T-level6_as_metric.T).mean(axis=1).mean()))
print(("level6 VS level5", (level6_as_metric.T-level5_as_metric.T).mean(axis=1).mean()))
print("\n")
print("\n")
print("\n")

# Per wave
for wave in all_waves:
    level5_mtr = eval(f"level5_as_metric").T[[f'{wave}_qtdb_single',f'{wave}_qtdb_multi',f'{wave}_ludb',f'{wave}_zhejiang',]].T
    level6_mtr = eval(f"level6_as_metric").T[[f'{wave}_qtdb_single',f'{wave}_qtdb_multi',f'{wave}_ludb',f'{wave}_zhejiang',]].T
    level7_mtr = eval(f"level7_as_metric").T[[f'{wave}_qtdb_single',f'{wave}_qtdb_multi',f'{wave}_ludb',f'{wave}_zhejiang',]].T
    print(f"########## {wave} wave ##########")
    print(("level7 VS level5", (level7_mtr.T-level5_mtr.T).mean(axis=1).mean()))
    print(("level7 VS level6", (level7_mtr.T-level6_mtr.T).mean(axis=1).mean()))
    print(("level6 VS level5", (level6_mtr.T-level5_mtr.T).mean(axis=1).mean()))
    print("\n")

print("\n")
print("\n")

# Per database
for db in all_databases:
    level5_mtr = level5_as_metric.T[[f"P_{db}",f"QRS_{db}",f"T_{db}"]].T
    level6_mtr = level6_as_metric.T[[f"P_{db}",f"QRS_{db}",f"T_{db}"]].T
    level7_mtr = level7_as_metric.T[[f"P_{db}",f"QRS_{db}",f"T_{db}"]].T
    print(f"########## {db} database ##########")
    print(("level7 VS level5", (level7_mtr.T-level5_mtr.T).mean(axis=1).mean()))
    print(("level7 VS level6", (level7_mtr.T-level6_mtr.T).mean(axis=1).mean()))
    print(("level6 VS level5", (level6_mtr.T-level5_mtr.T).mean(axis=1).mean()))
    print("\n")


('level7 VS level5', -0.1432638888888897)
('level7 VS level6', -0.2950694444444448)
('level6 VS level5', 0.12189814814814809)






########## P wave ##########
('level7 VS level5', -1.8368750000000011)
('level7 VS level6', -1.0029166666666676)
('level6 VS level5', -0.5227777777777778)


########## QRS wave ##########
('level7 VS level5', 0.15999999999999956)
('level7 VS level6', 0.04791666666666602)
('level6 VS level5', 0.06833333333333375)


########## T wave ##########
('level7 VS level5', 1.2470833333333327)
('level7 VS level6', 0.06979166666666714)
('level6 VS level5', 0.8201388888888883)






########## qtdb_single database ##########
('level7 VS level5', -0.46277777777778084)
('level7 VS level6', -0.6494444444444472)
('level6 VS level5', 0.18759259259259306)


########## qtdb_multi database ##########
('level7 VS level5', -0.34111111111111186)
('level7 VS level6', -0.4638888888888886)
('level6 VS level5', 0.1364814814814805)


########## zhejiang database ##########
('level7 VS

# Fix Zhejiang's and LUDB's mistake at computing on/off error

In [161]:
def get_ground_truth(basedir: str, database: str):
    if database == 'ludb':
        # Load data
        P = sak.load_data(os.path.join(basedir,"ludb","P.csv"))
        QRS = sak.load_data(os.path.join(basedir,"ludb","QRS.csv"))
        T = sak.load_data(os.path.join(basedir,"ludb","T.csv"))
    elif database == 'zhejiang':
        # Load data
        P = sak.load_data(os.path.join(basedir,"ZhejiangDB","P.csv"))
        QRS = sak.load_data(os.path.join(basedir,"ZhejiangDB","QRS.csv"))
        T = sak.load_data(os.path.join(basedir,"ZhejiangDB","T.csv"))
        
    # Divide into onsets/offsets
    Pon    = {k: P[k][0::2] for k in P}
    Poff   = {k: P[k][1::2] for k in P}
    QRSon  = {k: QRS[k][0::2] for k in QRS}
    QRSoff = {k: QRS[k][1::2] for k in QRS}
    Ton    = {k: T[k][0::2] for k in T}
    Toff   = {k: T[k][1::2] for k in T}

    # Generate validity
    validity = {
        k: [
            np.min(np.concatenate((P.get(k,[+np.inf]),QRS.get(k,[+np.inf]),T.get(k,[+np.inf])))),
            np.max(np.concatenate((P.get(k,[-np.inf]),QRS.get(k,[-np.inf]),T.get(k,[-np.inf])))),
        ] for k in QRS
    }
    return Pon,Poff,QRSon,QRSoff,Ton,Toff,validity
 
def get_file_list(basedir: str, database: str):
    if database == 'ludb':
        files = glob.glob(os.path.join(basedir,'ludb','*.dat'))
    elif database == 'zhejiang':
        files = glob.glob(os.path.join(basedir,'ZhejiangDB','RAW','*.csv'))
    return files
 
def get_sample(file: str, database: str):
    if database == 'ludb':
        (signal, header) = wfdb.rdsamp(os.path.splitext(file)[0])
        fs = header['fs']
    elif database == 'zhejiang':
        signal = pd.read_csv(file).values
        fs = 2000.
    return signal, fs


In [166]:
target_fs = {"ludb": 500, "zhejiang": 2000}

In [188]:
for folder in tqdm.tqdm(folders):
    if (
        (not os.path.isfile(os.path.join(folder,'metrics_string.txt'))) or
        (not os.path.isfile(os.path.join(folder,'zhejiang_metrics_string.txt'))) or
        (not os.path.isfile(os.path.join(folder,'ludb_metrics_string.txt'))) or
        (not os.path.isfile(os.path.join(folder,'metrics_fold_string.txt')))
       ):
        continue
    
    for db in ["zhejiang", "ludb"]:
        # Get predicted GT
        pon    = sak.load_data(os.path.join(folder,f"{db}_predicted_pon.csv"))
        poff   = sak.load_data(os.path.join(folder,f"{db}_predicted_poff.csv"))
        qrson  = sak.load_data(os.path.join(folder,f"{db}_predicted_qrson.csv"))
        qrsoff = sak.load_data(os.path.join(folder,f"{db}_predicted_qrsoff.csv"))
        ton    = sak.load_data(os.path.join(folder,f"{db}_predicted_ton.csv"))
        toff   = sak.load_data(os.path.join(folder,f"{db}_predicted_toff.csv"))

        # Get true fiducials
        Pon,Poff,QRSon,QRSoff,Ton,Toff,validity = get_ground_truth(basedir, db)

        # Produce metrics
        metrics = {}

        for wave in ['p','qrs','t']:
            metrics[wave] = {}
            metrics[wave]['truepositives'] = {}
            metrics[wave]['falsepositives'] = {}
            metrics[wave]['falsenegatives'] = {}
            metrics[wave]['onerrors'] = {}
            metrics[wave]['offerrors'] = {}

            input_on   = eval('{}on'.format(wave.lower()))
            input_off  = eval('{}off'.format(wave.lower()))
            target_on  = eval('{}on'.format(wave.upper()))
            target_off = eval('{}off'.format(wave.upper()))

            # Compute metrics
            for k_input in input_on:
                k_target = f"{k_input}###I"
                try:
                    # Refine input and output's regions w/ validity vectors
                    (input_on[k_input],input_off[k_input]) = src.metrics.filter_valid(input_on[k_input],input_off[k_input], validity[k_target][0], validity[k_target][1],operation="or")
                    (target_on[k_target],target_off[k_target]) = src.metrics.filter_valid(target_on[k_target],target_off[k_target], validity[k_target][0], validity[k_target][1],operation="or")
                    tp,fp,fn,dice,onerror,offerror = src.metrics.compute_metrics(input_on[k_input],input_off[k_input],target_on[k_target],target_off[k_target])
                except:
                    continue
                metrics[wave]['truepositives'][k_input] = tp
                metrics[wave]['falsepositives'][k_input] = fp
                metrics[wave]['falsenegatives'][k_input] = fn
                metrics[wave]['onerrors'][k_input] = np.copy(np.array(onerror))
                metrics[wave]['offerrors'][k_input] = np.copy(np.array(offerror))

        #########################################################################
        # Get stupid metric string
        metrics_string = ""
        metrics_string += "\n# {}".format(model_name)

        for wave in ['p','qrs','t']:
            metrics_string += "\n######### {} wave #########".format(wave.upper())
            metrics_string += "\n"
            metrics_string += "\nPrecision:    {}%".format(np.round(src.metrics.precision(sum(metrics[wave]['truepositives'].values()),sum(metrics[wave]['falsepositives'].values()),sum(metrics[wave]['falsenegatives'].values()))*100,decimals=2))
            metrics_string += "\nRecall:       {}%".format(np.round(src.metrics.recall(sum(metrics[wave]['truepositives'].values()),sum(metrics[wave]['falsepositives'].values()),sum(metrics[wave]['falsenegatives'].values()))*100,decimals=2))
            metrics_string += "\nF1 score:     {}%".format(np.round(src.metrics.f1_score(sum(metrics[wave]['truepositives'].values()),sum(metrics[wave]['falsepositives'].values()),sum(metrics[wave]['falsenegatives'].values()))*100,decimals=2))
            metrics_string += "\n"
            metrics_string += "\nOnset Error:  {} ± {} ms".format(np.round(np.mean([v for l in metrics[wave]['onerrors'].values() for v in l])/target_fs[db]*1000,decimals=2),np.round(np.std([v for l in metrics[wave]['onerrors'].values() for v in l])/target_fs[db]*1000,decimals=2))
            metrics_string += "\nOffset Error: {} ± {} ms".format(np.round(np.mean([v for l in metrics[wave]['offerrors'].values() for v in l])/target_fs[db]*1000,decimals=2),np.round(np.std([v for l in metrics[wave]['offerrors'].values() for v in l])/target_fs[db]*1000,decimals=2))
            metrics_string += "\n\n"

        metrics_string += "\n---"

        
        # Save produced metrics
        original_stdout = sys.stdout # Save a reference to the original standard output
        with open(os.path.join(folder,f'{db}_metrics_string.txt'), 'w') as f:
            sys.stdout = f # Change the standard output to the file we created.
            print(metrics_string)

        sys.stdout = original_stdout # Reset the standard output to its original value


100%|██████████| 109/109 [00:42<00:00,  2.58it/s]


In [170]:
# Get predicted GT
pon    = sak.load_data(os.path.join(folder,f"{db}_predicted_pon.csv"))
poff   = sak.load_data(os.path.join(folder,f"{db}_predicted_poff.csv"))
qrson  = sak.load_data(os.path.join(folder,f"{db}_predicted_qrson.csv"))
qrsoff = sak.load_data(os.path.join(folder,f"{db}_predicted_qrsoff.csv"))
ton    = sak.load_data(os.path.join(folder,f"{db}_predicted_ton.csv"))
toff   = sak.load_data(os.path.join(folder,f"{db}_predicted_toff.csv"))

# Get true fiducials
Pon,Poff,QRSon,QRSoff,Ton,Toff,validity = get_ground_truth(basedir, 'zhejiang')

In [171]:
# Produce metrics
metrics = {}

for wave in ['p','qrs','t']:
    metrics[wave] = {}
    metrics[wave]['truepositives'] = {}
    metrics[wave]['falsepositives'] = {}
    metrics[wave]['falsenegatives'] = {}
    metrics[wave]['onerrors'] = {}
    metrics[wave]['offerrors'] = {}

    input_on   = eval('{}on'.format(wave.lower()))
    input_off  = eval('{}off'.format(wave.lower()))
    target_on  = eval('{}on'.format(wave.upper()))
    target_off = eval('{}off'.format(wave.upper()))

    # Compute metrics
    for k_input in input_on:
        k_target = f"{k_input}###I"
        try:
            # Refine input and output's regions w/ validity vectors
            (input_on[k_input],input_off[k_input]) = src.metrics.filter_valid(input_on[k_input],input_off[k_input], validity[k_target][0], validity[k_target][1],operation="or")
            (target_on[k_target],target_off[k_target]) = src.metrics.filter_valid(target_on[k_target],target_off[k_target], validity[k_target][0], validity[k_target][1],operation="or")
            tp,fp,fn,dice,onerror,offerror = src.metrics.compute_metrics(input_on[k_input],input_off[k_input],target_on[k_target],target_off[k_target])
        except:
            continue
        metrics[wave]['truepositives'][k_input] = tp
        metrics[wave]['falsepositives'][k_input] = fp
        metrics[wave]['falsenegatives'][k_input] = fn
        metrics[wave]['onerrors'][k_input] = np.copy(np.array(onerror))
        metrics[wave]['offerrors'][k_input] = np.copy(np.array(offerror))


In [180]:
#########################################################################
# Get stupid metric string
metrics_string = ""
metrics_string += "\n# {}".format(model_name)

for wave in ['p','qrs','t']:
    metrics_string += "\n######### {} wave #########".format(wave.upper())
    metrics_string += "\n"
    metrics_string += "\nPrecision:    {}%".format(np.round(src.metrics.precision(sum(metrics[wave]['truepositives'].values()),sum(metrics[wave]['falsepositives'].values()),sum(metrics[wave]['falsenegatives'].values()))*100,decimals=2))
    metrics_string += "\nRecall:       {}%".format(np.round(src.metrics.recall(sum(metrics[wave]['truepositives'].values()),sum(metrics[wave]['falsepositives'].values()),sum(metrics[wave]['falsenegatives'].values()))*100,decimals=2))
    metrics_string += "\nF1 score:     {}%".format(np.round(src.metrics.f1_score(sum(metrics[wave]['truepositives'].values()),sum(metrics[wave]['falsepositives'].values()),sum(metrics[wave]['falsenegatives'].values()))*100,decimals=2))
    metrics_string += "\n"
    metrics_string += "\nOnset Error:  {} ± {} ms".format(np.round(np.mean([v for l in metrics[wave]['onerrors'].values() for v in l])/target_fs[db]*1000,decimals=2),np.round(np.std([v for l in metrics[wave]['onerrors'].values() for v in l])/target_fs[db]*1000,decimals=2))
    metrics_string += "\nOffset Error: {} ± {} ms".format(np.round(np.mean([v for l in metrics[wave]['offerrors'].values() for v in l])/target_fs[db]*1000,decimals=2),np.round(np.std([v for l in metrics[wave]['offerrors'].values() for v in l])/target_fs[db]*1000,decimals=2))
    metrics_string += "\n\n"

metrics_string += "\n---"


In [183]:
db

'zhejiang'

In [182]:
os.path.join(folder,f'{db}_metrics_string.txt'),

'/media/guille/DADES/DADES/Delineator/TrainedModels/UNet5LevelsConvDiceOnly_real_20201204204042'

In [184]:
shutil.copy(os.path.join(folder,f'{db}_metrics_string.txt'),os.path.join(folder,f'{db}_metrics_string_pre_fix.txt'))

'/media/guille/DADES/DADES/Delineator/TrainedModels/UNet5LevelsConvDiceOnly_real_20201204204042/zhejiang_metrics_string_pre_fix.txt'

In [181]:
print(metrics_string)


# UNet5LevelsConvDiceOnly_synth_20201211065325
######### P wave #########

Precision:    98.35%
Recall:       94.96%
F1 score:     96.62%

Onset Error:  3.3 ± 14.38 ms
Offset Error: 9.79 ± 12.63 ms


######### QRS wave #########

Precision:    99.13%
Recall:       99.66%
F1 score:     99.4%

Onset Error:  -0.29 ± 13.13 ms
Offset Error: 4.82 ± 11.68 ms


######### T wave #########

Precision:    94.85%
Recall:       98.85%
F1 score:     96.81%

Onset Error:  0.94 ± 34.25 ms
Offset Error: -3.23 ± 35.53 ms


---


# TMP

In [5]:
def get_ground_truth(basedir: str, database: str):
    if database == 'ludb':
        # Load data
        P = sak.load_data(os.path.join(basedir,"ludb","P.csv"))
        QRS = sak.load_data(os.path.join(basedir,"ludb","QRS.csv"))
        T = sak.load_data(os.path.join(basedir,"ludb","T.csv"))
    elif database == 'zhejiang':
        # Load data
        P = sak.load_data(os.path.join(basedir,"ZhejiangDB","P.csv"))
        QRS = sak.load_data(os.path.join(basedir,"ZhejiangDB","QRS.csv"))
        T = sak.load_data(os.path.join(basedir,"ZhejiangDB","T.csv"))
        
    # Divide into onsets/offsets
    Pon    = {k: P[k][0::2] for k in P}
    Poff   = {k: P[k][1::2] for k in P}
    QRSon  = {k: QRS[k][0::2] for k in QRS}
    QRSoff = {k: QRS[k][1::2] for k in QRS}
    Ton    = {k: T[k][0::2] for k in T}
    Toff   = {k: T[k][1::2] for k in T}

    # Generate validity
    validity = {
        k: [
            np.min(np.concatenate((P.get(k,[+np.inf]),QRS.get(k,[+np.inf]),T.get(k,[+np.inf])))),
            np.max(np.concatenate((P.get(k,[-np.inf]),QRS.get(k,[-np.inf]),T.get(k,[-np.inf])))),
        ] for k in QRS
    }
    return Pon,Poff,QRSon,QRSoff,Ton,Toff,validity
 
def get_file_list(basedir: str, database: str):
    if database == 'ludb':
        files = glob.glob(os.path.join(basedir,'ludb','*.dat'))
    elif database == 'zhejiang':
        files = glob.glob(os.path.join(basedir,'ZhejiangDB','RAW','*.csv'))
    return files
 
def get_sample(file: str, database: str):
    if database == 'ludb':
        (signal, header) = wfdb.rdsamp(os.path.splitext(file)[0])
        fs = header['fs']
    elif database == 'zhejiang':
        signal = pd.read_csv(file).values
        fs = 2000.
    return signal, fs


In [7]:
basedir = '/media/guille/DADES/DADES/Delineator/'
_,_,QRSon,QRSoff,_,_,_ = get_ground_truth(basedir,"ludb")

In [32]:
QRSon  = sak.load_data(os.path.join(basedir, 'QTDB', 'QRSonNew.csv'))

In [36]:
sum([QRSon[k].size for k in QRSon])//2

3295

In [8]:
counter = 0

for k in QRSon:
    if k.endswith("###I"):
        counter += QRSon[k].size

In [30]:
counter = 0

for file in glob.glob('/media/guille/DADES/DADES/PhysioNet/QTDB/RAW/*.q1c'):
    fname,ext = os.path.splitext(file)
    ext = ext.replace('.','')
    
    ann = wfdb.rdann(fname,ext)
    counter += (np.array(ann.symbol) == 'N').sum()


In [31]:
counter

3528

In [16]:
21966//12

1830

In [12]:
counter

611

In [393]:
file_list = get_file_list(basedir,"zhejiang")

In [399]:
samples = []

for file in file_list:
    sample,fs = get_sample(file,"zhejiang")
    
    samples.append(sample.shape[0])

In [403]:
min(samples)

5791

In [407]:
max(samples)/fs

22.622